In [1]:
%cd ../../../..
%pwd

/Users/jbbutler129/Google Drive (butlerj@berkeley.edu)/Classes/215A/final_project/rule-vetting


'/Users/jbbutler129/Google Drive (butlerj@berkeley.edu)/Classes/215A/final_project/rule-vetting'

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import rulevetting.api.viz as viz
from rulevetting.projects.tbi_pecarn.dataset import Dataset

outcome_def = 'outcome'  # output

/opt/anaconda3/lib/python3.8/site-packages/redis/connection.py:77: UserWarning: redis-py works best with hiredis. Please consider installing
  warnings.warn(msg)


## Loading the Data

In [3]:
df_train, df_tune, df_test = Dataset().get_data()

kwargs {'clean_data': {}, 'preprocess_data': {'step1_injMech': False, 'step5_missSubGCS': True, 'step5_fake15GCS': True, 'step5_fake14GCS': True, 'step8_missingOSI': True, 'step9_HEMA': 3, 'step10_cautiousUncl': True, 'step14_vomitDtls': False, 'step15_HA': 2, 'step15_HAStart': False, 'step16_Seiz': 2, 'step16_SeizOccur': False, 'step17_cautiousUncl': True, 'step19_Drugs': False, 'step20_ActNormal': True}, 'extract_features': {'HEMA_umbrella': False, 'SFxPalp_umbrella': False, 'SFxBas_umbrella': False, 'Clav_umbrella': False, 'NeuroD_umbrella': False, 'Vomit_umbrella': False, 'HA_umbrella': False, 'Seiz_umbrella': False, 'LOC_umbrella': False, 'remove_constVal': True}}


In [9]:
df_train

,LOCSeparate,Seiz,ActNorm,Vomit,GCSEye,GCSVerbal,GCSMotor,GCSTotal,AMS,SFxPalp,...,OSIFlank_92,OSIAbdomen_0,OSIAbdomen_1,OSIAbdomen_92,OSIPelvis_0,OSIPelvis_1,OSIPelvis_92,OSIOth_0,OSIOth_1,OSIOth_92
PatNum,,,,,,,,,,,,,,,,,,,,,
1788,0.0,0.0,1.0,0.0,4.0,5.0,6.0,15,0.0,0.0,...,1,0,0,1,0,0,1,0,0,1
11823,0.0,0.0,1.0,0.0,4.0,5.0,6.0,15,0.0,0.0,...,1,0,0,1,0,0,1,0,0,1
41601,0.0,0.0,1.0,0.0,4.0,5.0,6.0,15,0.0,0.0,...,1,0,0,1,0,0,1,0,0,1
22608,0.0,0.0,1.0,0.0,4.0,5.0,6.0,15,0.0,0.0,...,1,0,0,1,0,0,1,0,0,1
2980,0.0,0.0,1.0,0.0,4.0,5.0,6.0,15,0.0,0.0,...,1,0,0,1,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9669,0.0,0.0,1.0,0.0,4.0,5.0,6.0,15,0.0,0.0,...,1,0,0,1,0,0,1,0,0,1
31876,0.0,0.0,1.0,0.0,4.0,5.0,6.0,15,0.0,0.0,...,1,0,0,1,0,0,1,0,0,1
11121,0.0,0.0,1.0,0.0,4.0,5.0,6.0,15,0.0,0.0,...,1,0,0,1,0,0,1,0,0,1


In [ ]:
plt.figure(dpi=250, figsize=(4, 7))
vals = corrs[outcome_def]
args = np.argsort(vals)
labs = vals.index.values[args]
ax = plt.subplot(111)
plt.barh(labs[:-1], vals[args][:-1])
plt.xlabel('Correlation w/ outcome')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)